In [1]:
import numpy as np
import pandas as pd
import os
import re
import glob
import csv
DC_311_2020 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2020.csv'
DC_311_2019 = '/Users/julia/Box/Data_311/DMV_311/DMV_nonEM/DC/311_Washington_DC_2019.csv'

### All 311 Requests in D.C in 2019

In [2]:
DC_311_2019_df = pd.read_csv(DC_311_2019)
DC_311_2019_df

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (12,14,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,-76.930018,38.904475,278741,S0031,Bulk Collection,SWMA- Solid Waste Management Admistration,DPW,1,2019/07/18 00:57:15+00,2019/08/05 19:42:09+00,...,406070.32,137403.48,38.904467,-76.930016,WASHINGTON,DC,20019.0,15976.0,7.0,NaN
1,-77.034081,38.941341,278742,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019/07/18 01:00:00+00,2019/07/21 03:03:10+00,...,397045.59,141494.19,38.941334,-77.034079,WASHINGTON,DC,20011.0,259155.0,4.0,NaN
2,-77.034326,38.915358,278743,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2019/07/18 01:58:52+00,2019/07/19 07:23:15+00,...,397023.24,138609.83,38.915350,-77.034324,WASHINGTON,DC,20009.0,240017.0,2.0,NaN
3,-76.942364,38.898615,278744,S0336,Out of State Parking Violation (ROSA),PEMA- Parking Enforcement Management Administr...,DPW,1,2019/07/18 01:59:10+00,2019/07/18 06:39:36+00,...,404999.88,136752.30,38.898608,-76.942362,WASHINGTON,DC,20019.0,25746.0,7.0,NaN
4,-76.978173,38.888846,278745,S0316,Recycling Cart Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2019/07/18 01:01:44+00,2019/08/19 11:09:05+00,...,401893.88,135666.52,38.888839,-76.978170,WASHINGTON,DC,20003.0,55558.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365006,-77.017162,38.877622,722564,VACPROP,DCRA - Vacant Private Property Inspection,Dept of Consumer & Regulatory Affairs,DCRA,1,2019/02/21 02:46:15+00,2019/03/18 18:35:59+00,...,398511.05,134420.39,38.877614,-77.017160,WASHINGTON,DC,20024.0,307170.0,6.0,NaN
365007,-77.017162,38.877622,722565,ILLEGALCON,DCRA - Illegal Construction,Dept of Consumer & Regulatory Affairs,DCRA,1,2019/02/22 12:08:40+00,2019/04/02 16:07:35+00,...,398511.05,134420.39,38.877614,-77.017160,WASHINGTON,DC,20024.0,307170.0,6.0,After review of the submitted service request ...
365008,-77.017162,38.877622,722566,VACPROP,DCRA - Vacant Private Property Inspection,Dept of Consumer & Regulatory Affairs,DCRA,1,2019/03/25 20:37:36+00,2019/03/25 21:02:02+00,...,398511.05,134420.39,38.877614,-77.017160,WASHINGTON,DC,20024.0,307170.0,6.0,NaN
365009,-77.017162,38.877622,722567,VACPROP,DCRA - Vacant Private Property Inspection,Dept of Consumer & Regulatory Affairs,DCRA,1,2019/04/02 15:57:01+00,2019/04/02 17:18:43+00,...,398511.05,134420.39,38.877614,-77.017160,WASHINGTON,DC,20024.0,307170.0,6.0,NaN


In [4]:
DC_311_2019_df['STATUS_CODE'].unique()

array([nan, 'CLOSED', 'OPEN'], dtype=object)

In [3]:
print(DC_311_2019_df.columns.tolist())
print(len(DC_311_2019_df.columns.tolist()))

['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION', 'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT', 'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE', 'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME', 'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY', 'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY', 'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS']
30


In [4]:
DC_311_2019_df.ORGANIZATIONACRONYM.unique().tolist()

['DPW',
 'DDOT',
 'DOEE',
 'DOH',
 'FEMS',
 'DMV',
 'ORM',
 'OUC',
 'DCRA',
 nan,
 'DDS',
 'DGS',
 'DFHV',
 '311 Customer Service Center']

In [5]:
DC_311_2019_fems = DC_311_2019_df.loc[DC_311_2019_df['ORGANIZATIONACRONYM'] == 'FEMS']
DC_311_2019_fems

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
91,-76.925117,38.886951,279126,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/04 17:06:41+00,2019/01/07 15:06:27+00,...,406497.01,135458.59,38.886944,-76.925115,WASHINGTON,DC,20019.0,10428.0,7.0,NaN
539,-76.960797,38.877512,279574,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/16 22:27:57+00,2019/03/23 16:02:28+00,...,403401.88,134408.80,38.877504,-76.960795,WASHINGTON,DC,20019.0,50666.0,7.0,FIREFIGHTERS NEVER SHOWED UP.
707,-76.981361,38.922836,279742,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/18 19:50:06+00,2019/01/23 21:05:37+00,...,401616.50,139439.57,38.922828,-76.981359,WASHINGTON,DC,20018.0,61311.0,5.0,NaN
1051,-77.020444,38.961513,280086,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/15 18:43:00+00,2019/02/01 19:23:58+00,...,398228.31,143733.09,38.961505,-77.020442,WASHINGTON,DC,20011.0,246463.0,4.0,REQUEST SMOKE ALARMS.
1187,-76.993583,38.892218,280222,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/01/11 19:02:21+00,2019/03/14 17:30:29+00,...,400556.86,136040.64,38.892211,-76.993581,WASHINGTON,DC,20002.0,81651.0,6.0,"SMOKE ALARMS, INSTALLED."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359992,-77.022480,38.930865,716835,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/26 14:17:21+00,2020/01/17 20:15:20+00,...,398051.03,140330.95,38.930858,-77.022478,WASHINGTON,DC,20010.0,228942.0,1.0,NaN
360159,-77.000487,38.962512,717002,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/26 21:31:56+00,2020/01/03 14:58:46+00,...,399957.95,143843.85,38.962505,-77.000485,WASHINGTON,DC,20011.0,25978.0,4.0,NaN
363225,-77.019651,38.958943,720116,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2019/12/30 19:19:01+00,2020/01/27 19:52:12+00,...,398297.04,143447.84,38.958936,-77.019648,WASHINGTON,DC,20011.0,248819.0,4.0,NaN
363470,-76.995047,38.853735,720361,FEMSFIREINSP,FEMS - Fire Safety Inspection,FEMS,FEMS,1,2019/12/30 21:40:15+00,2019/12/30 22:22:11+00,...,400430.11,131768.69,38.853728,-76.995045,WASHINGTON,DC,20032.0,301360.0,8.0,NaN


### All 311 Requests in D.C in 2020

In [6]:
DC_311_2020_df = pd.read_csv(DC_311_2020)
DC_311_2020_df

/opt/anaconda3/envs/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
0,-77.052255,38.901881,1,S05SL,Streetlight Repair Investigation,Transportation Operations Administration,DDOT,1,2020/01/01 05:00:38+00,2020/01/04 08:35:04+00,...,395467.53,137114.59,38.901874,-77.052253,WASHINGTON,DC,20037.0,273983.0,2.0,NaN
1,-76.995047,38.853735,2,S0311,Rodent Inspection and Treatment,DOH- Department Of Health,DOH,1,2020/01/01 05:02:57+00,2020/01/01 05:12:17+00,...,400430.11,131768.69,38.853728,-76.995045,WASHINGTON,DC,20032.0,301360.0,8.0,NaN
2,-77.024210,38.913582,3,S0441,Trash Collection - Missed,SWMA- Solid Waste Management Admistration,DPW,1,2020/01/01 05:20:59+00,2020/01/02 13:08:05+00,...,397900.54,138412.45,38.913575,-77.024208,WASHINGTON,DC,20001.0,239705.0,6.0,NaN
3,-77.025296,38.905826,4,S0441,Trash Collection - Missed,SWMA- Solid Waste Management Admistration,DPW,1,2020/01/01 06:34:21+00,2020/01/02 13:05:05+00,...,397806.12,137551.45,38.905818,-77.025294,WASHINGTON,DC,20001.0,239755.0,2.0,NaN
4,-77.014662,38.881105,5,EMNPV,Emergency No-Parking Verification,PEMA- Parking Enforcement Management Administr...,DPW,1,2020/01/01 09:07:20+00,2020/01/01 09:18:13+00,...,398728.03,134807.00,38.881097,-77.014660,WASHINGTON,DC,20024.0,277827.0,6.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304745,-76.992409,38.882031,659524,S0422,Supercan - Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2020/12/23 12:40:13+00,2021/01/13 14:43:16+00,...,400658.86,134909.78,38.882024,-76.992406,WASHINGTON,DC,20003.0,12224.0,6.0,NaN
304746,-77.013005,38.915761,659841,S0451,Trash Cart - Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2020/12/22 20:00:59+00,2021/01/28 13:57:34+00,...,398872.32,138654.15,38.915754,-77.013003,WASHINGTON,DC,20001.0,227465.0,5.0,NaN
304747,-76.973669,38.896967,659842,TRACO001,Trash Cart Repair,SWMA- Solid Waste Management Admistration,DPW,1,2020/11/09 14:00:51+00,2021/02/04 13:22:26+00,...,402284.35,136568.07,38.896959,-76.973667,WASHINGTON,DC,20002.0,10348.0,7.0,NaN
304748,-77.057243,38.919885,659843,S0451,Trash Cart - Delivery,SWMA- Solid Waste Management Admistration,DPW,1,2020/10/14 16:01:01+00,2021/02/05 19:52:36+00,...,395036.12,139113.39,38.919878,-77.057241,WASHINGTON,DC,20008.0,220401.0,3.0,NaN


In [7]:
print(DC_311_2020_df.columns.tolist())
print(len(DC_311_2020_df.columns.tolist()))

['X', 'Y', 'OBJECTID', 'SERVICECODE', 'SERVICECODEDESCRIPTION', 'SERVICETYPECODEDESCRIPTION', 'ORGANIZATIONACRONYM', 'SERVICECALLCOUNT', 'ADDDATE', 'RESOLUTIONDATE', 'SERVICEDUEDATE', 'SERVICEORDERDATE', 'INSPECTIONFLAG', 'INSPECTIONDATE', 'INSPECTORNAME', 'SERVICEORDERSTATUS', 'STATUS_CODE', 'SERVICEREQUESTID', 'PRIORITY', 'STREETADDRESS', 'XCOORD', 'YCOORD', 'LATITUDE', 'LONGITUDE', 'CITY', 'STATE', 'ZIPCODE', 'MARADDRESSREPOSITORYID', 'WARD', 'DETAILS']
30


In [8]:
DC_311_2020_fems = DC_311_2020_df.loc[DC_311_2020_df['ORGANIZATIONACRONYM'] == 'FEMS']
DC_311_2020_fems

,X,Y,OBJECTID,SERVICECODE,SERVICECODEDESCRIPTION,SERVICETYPECODEDESCRIPTION,ORGANIZATIONACRONYM,SERVICECALLCOUNT,ADDDATE,RESOLUTIONDATE,...,XCOORD,YCOORD,LATITUDE,LONGITUDE,CITY,STATE,ZIPCODE,MARADDRESSREPOSITORYID,WARD,DETAILS
4948,-77.000547,38.962513,6405,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/01/04 23:27:47+00,2020/01/16 20:52:53+00,...,399952.76,143843.87,38.962505,-77.000545,WASHINGTON,DC,20011.0,26070.0,4.0,NaN
5495,-76.935298,38.902693,7208,FEMEDUPR,FEMS - Fire Safety Education,FEMS,FEMS,1,2020/01/06 12:29:45+00,2020/02/20 14:39:52+00,...,405612.45,137205.37,38.902686,-76.935296,WASHINGTON,DC,20019.0,18109.0,7.0,NaN
5561,-76.988792,38.835090,7274,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/01/06 14:17:17+00,2020/01/27 19:53:13+00,...,400973.34,129699.06,38.835083,-76.988789,WASHINGTON,DC,20032.0,279187.0,8.0,NaN
6401,-76.936911,38.880609,8306,FESPEDNU,FEMS - Community Events,FEMS-Special Events,FEMS,1,2020/01/06 18:58:23+00,2020/01/10 19:00:26+00,...,405474.30,134753.82,38.880602,-76.936908,WASHINGTON,DC,20019.0,21041.0,7.0,NaN
6471,-76.991575,38.903698,8376,FESPEDNU,FEMS - Community Events,FEMS-Special Events,FEMS,1,2020/01/06 17:49:35+00,2020/01/09 15:13:11+00,...,400730.97,137315.00,38.903690,-76.991573,WASHINGTON,DC,20002.0,285821.0,5.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79629,-76.993517,38.881809,111901,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/03/27 18:59:13+00,2020/04/09 13:54:48+00,...,400562.70,134885.09,38.881801,-76.993515,WASHINGTON,DC,20003.0,10968.0,6.0,NaN
80583,-77.004331,38.914935,113639,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/03/29 03:02:57+00,2020/04/09 13:54:48+00,...,399624.61,138562.33,38.914927,-77.004328,WASHINGTON,DC,20002.0,296345.0,5.0,NaN
84250,-77.021938,38.957644,118682,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/04/03 05:15:51+00,2020/04/09 13:54:48+00,...,398098.71,143303.60,38.957636,-77.021936,WASHINGTON,DC,20011.0,246643.0,4.0,NaN
88092,-76.990267,38.950702,125308,FESAADNU,FEMS - Smoke Alarm Application,FEMS-Smoke Alarms,FEMS,1,2020/04/08 18:51:25+00,2020/04/09 13:54:48+00,...,400843.90,142532.81,38.950694,-76.990264,WASHINGTON,DC,20017.0,15424.0,5.0,NaN


In [9]:
print("#311 reports that were assigned to FEMS in 2019: " + str(DC_311_2019_fems.shape[0]))
print("#311 reports that were assigned to FEMS in 2020: " + str(DC_311_2020_fems.shape[0]))

print("FEMS reports/ total #311 in 2019 " + str(DC_311_2019_fems.shape[0] / DC_311_2019_df.shape[0]))
print("FEMS reports/ total #311 in 2020 " + str(DC_311_2020_fems.shape[0] / DC_311_2020_df.shape[0]))

#311 reports that were assigned to FEMS in 2019: 1254
#311 reports that were assigned to FEMS in 2020: 160
FEMS reports/ total #311 in 2019 0.003435512902350888
FEMS reports/ total #311 in 2020 0.0005250205086136178


In [10]:
DC_311_2019_fems['dates'] = pd.to_datetime(DC_311_2019_fems['ADDDATE'], format='%Y/%m/%d')

<ipython-input-10-35b38d23c2bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DC_311_2019_fems['dates'] = pd.to_datetime(DC_311_2019_fems['ADDDATE'], format='%Y/%m/%d')


In [11]:
DC_311_2019_fems['add_month'] = DC_311_2019_fems['dates'].dt.month
DC_311_2019_fems.groupby('add_month').size()

<ipython-input-11-972359df5f8d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DC_311_2019_fems['add_month'] = DC_311_2019_fems['dates'].dt.month


add_month
1     124
2      77
3     104
4     126
5     119
6     125
7     111
8     105
9     102
10    131
11     79
12     51
dtype: int64

In [12]:
DC_311_2020_fems['dates'] = pd.to_datetime(DC_311_2020_fems['ADDDATE'], format='%Y/%m/%d')
DC_311_2020_fems['add_month'] = DC_311_2020_fems['dates'].dt.month
DC_311_2020_fems.groupby('add_month').size()

<ipython-input-12-de5183375514>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DC_311_2020_fems['dates'] = pd.to_datetime(DC_311_2020_fems['ADDDATE'], format='%Y/%m/%d')
<ipython-input-12-de5183375514>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DC_311_2020_fems['add_month'] = DC_311_2020_fems['dates'].dt.month


add_month
1    55
2    72
3    30
4     3
dtype: int64